<div align="center">
<h1>25 Pandas Coding Mistakes You Should Avoid</h1>

</div>
<div align="center">
Link for the original video: https://youtu.be/_gaAoJBMJ_Q
</div>

### 1. Writing into csv with unnecessary index column

In [2]:
import pandas as pd
df = pd.read_csv('housing.csv')

# Using `index=False` avoids adding unnecassary 
# unnamed index column to the output csv file 
df.to_csv('output.csv', index=False)

### 2. Using column names that include spaces

In [ ]:
# It's preferable to avoid including spaces 
# in column names And use underscores instead
df['bedroom_percentage'] = df['total_bedrooms'] / df['total_rooms']

# The column becomes accessible via the dot synatx
df.bedroom_percentage

In [ ]:
# And querying it, is now much easier
df.query('bedroom_percentage >= 0.7')

### 3. Not leveraging the QUERY method

In [ ]:
df.query('housing_median_age > 50 and median_income < 1')

### 4.  Formulating query strings using string methods

<div align="center">
pandas queries can access external variables by simply using the `@` symbol before the variable instead of formatting them via string methods
</div>

In [ ]:
min_pop = 1000
min_income = 0.7

df.query('population > @min_pop and median_income > @min_income')

### 5. Using the `inplace=True` paramter.

In [ ]:
# It's preferable to explicitly overwrite modifications 
# As "inplace" is generally frowned upon and 
# Also could be removed in future versions.

df = df.fillna(0)

### 6. Iterating over the rows when vectorization is an option

In [ ]:
# Bad practice
for i, row in df.iterrows():
    if row['housing_median_age'] > 50:
        df.loc[i, 'is_old'] = True
    else:
        df.loc[i, 'is_old'] = False
        

In [ ]:
# Good practice
df['is_old'] = df['housing_median_age'] > 50
df

### 7. Using the `apply` method when vectorization is an option

In [ ]:
# Bad practice
df['population_squared'] = df.apply(lambda row: row['population'] ** 2, axis=1)

In [ ]:
# Good practice
df['population_squared'] = df['population']**2

### 8. Treating a slice of a dataframe as if it was a new dataframe

In [ ]:
# Bad practice
netflix_df = pd.read_csv('Netflix.csv')
netflix_df_new = netflix_df.query('release_year > 2010')

netflix_df_new['director_first_name'] = netflix_df_new['director'].str[-5:]
netflix_df_new

In [ ]:
# Good practice
netflix_df_new = netflix_df.query('release_year > 2010').copy()

netflix_df_new['director_first_name'] = netflix_df_new['director'].str[-5:]
netflix_df_new

### 9. Creating multiple intermediate dataframes when transforming a dataframe

In [ ]:
# Bad practice
netflix_df_out = netflix_df.query('release_year > 2010')
netflix_df_out2 = netflix_df_out.groupby(['rating'])[['duration']].mean()
netflix_df_out3 = netflix_df_out2.sort_values(by='duration')

netflix_df_out3

In [ ]:
# Good practice

netflix_df_out = (netflix_df.query('release_year > 2010').
    groupby(['rating'])[['duration']].mean().sort_values(by='duration'))

netflix_df_out

### 10. Not properly setting columns dtypes

In [ ]:
# Each column in a pandas dataframe has a dtype
# When reading in a csv file, pandas will try to infer the dtype
# But sometimes it's not possible to infer the dtype correctly
# In such cases, we can explicitly specify the dtype

# Dates can be specified when reading in the csv file
netflix_df = pd.read_csv('Netflix.csv', parse_dates=['date_added'])

# Or after the csv file has been read in
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'])

### 11. Using a string value instead of a boolean

In [ ]:
# Bad practice
netflix_df['after_2010'] = 'NO'
netflix_df.loc[netflix_df['release_year'] > 2010, 'after_2010'] = 'YES'
netflix_df

# Manual fix is to map the values
netflix_df['after_2010'] = netflix_df['after_2010'].map({'YES': True, 'NO': False})

In [ ]:
# Good practice
netflix_df['before_2010'] = netflix_df['release_year'] <= 2010
netflix_df


### 12. Not leveraging pandas built-in plotting methods

In [ ]:
ax = netflix_df.plot(kind='scatter', x='release_year', y='duration', figsize=(10, 6))

### 13. Manually applying string methods

In [ ]:
# Bad practice
netflix_df['director_first_name'] = netflix_df['director'].apply(lambda x: str(x).upper())

In [ ]:
#Good practice
netflix_df['director_first_name'] = netflix_df['director'].str.upper()

### 14. Repeating commonly used data transformations

In [ ]:
# It's best practice to write pipelines as functions

# Example pipeline
def process_data(df):
    df['Time_Normalized'] = df['Time'] / df['Time'].mean()
    df['Place'] = df['Place'].str.lower()
    return df

dfw = pd.read_csv('data.csv')
dfw = process_data(dfw)
dfw

### 15. Manually renaming columns

In [ ]:
# Bad practice

df = pd.DataFrame({'Time': [1, 2, 3], 'Place': ['A', 'B', 'C'], 'Value': [10, 20, 30]})
df.columns = ['time', 'place', 'value']

In [ ]:
# Good practice

df = pd.DataFrame({'Time': [1, 2, 3], 'Place': ['A', 'B', 'C'], 'Value': [10, 20, 30]})
df = df.rename(columns={'Time': 'time', 'Place': 'place', 'Value': 'value'})

### 16. Aggregating by groups manually

In [4]:
# Bad practice

df = pd.read_csv('Netflix.csv')
tv_shows = df.loc[df['type'] == 'TV Show']['duration'].mean()
movies = df.loc[df['type'] == 'Movie']['duration'].mean()
print(tv_shows, movies)

1.7759336099585061 99.30797842663195


In [5]:
# Good practice

df = pd.read_csv('Netflix.csv')
df.groupby('type')['duration'].mean()

type
Movie      99.307978
TV Show     1.775934
Name: duration, dtype: float64

### 17. Looping to create aggregates instead of using `grouby().agg()` method

In [8]:
# Bad practice

import numpy as np

# Loop over the rows of the dataframe to apppend the durations to a list
durations_tv = []
durations_movies = []

for i, row in df.iterrows():
    if row['type'] == 'TV Show':
        durations_tv.append(int(row['duration']))
    else:
        durations_movies.append(int(row['duration']))
        
# Convert the list to a numpy array
durations_tv = np.array(durations_tv)
durations_movies = np.array(durations_movies)

# Calculate the mean
tv_shows = np.mean(durations_tv)
movies = np.mean(durations_movies)

print('Mean duration of TV shows: ', tv_shows)
print('Mean duration of movies: ', movies)

# Calculate the median
tv_shows = np.median(durations_tv)
movies = np.median(durations_movies)

print('Median duration of TV shows: ', tv_shows)
print('Median duration of movies: ', movies)

# Calculate the count
tv_shows = np.count_nonzero(durations_tv)
movies = np.count_nonzero(durations_movies)

print('Count of TV shows: ', tv_shows)
print('Count of movies: ', movies)


Mean duration of TV shows:  1.7759336099585061
Mean duration of movies:  99.30797842663195
Median duration of TV shows:  1.0
Median duration of movies:  98.0
Count of TV shows:  2410
Count of movies:  5377


In [6]:
# Good practice

df = pd.read_csv('Netflix.csv')
df.groupby('type')['duration'].agg(['mean', 'median', 'count'])

,mean,median,count
type,,,
Movie,99.307978,98.0,5377
TV Show,1.775934,1.0,2410


### 18. Looping to calculate how a value changes

In [9]:
# Bad practice

df = pd.read_csv('housing.csv')

for i in range(1, len(df)):
    df.loc[i, 'median_house_value_change'] = (
        (df.loc[i, 'median_house_value'] - df.loc[i-1, 'median_house_value']) / df.loc[i-1, 'median_house_value'])
    df.loc[i, 'median_house_value_diff'] = df.loc[i, 'median_house_value'] - df.loc[i-1, 'median_house_value']

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,median_house_value_change,median_house_value_diff
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0,NaN,NaN
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0,0.197309,13200.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0,0.069913,5600.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,-0.143524,-12300.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0,-0.107629,-7900.0


In [10]:
# Good practice

df = pd.read_csv('housing.csv')

df['pct_change'] = df['median_house_value'].pct_change()
df['diff'] = df['median_house_value'].diff()

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,pct_change,diff
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0,NaN,NaN
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0,0.197309,13200.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0,0.069913,5600.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,-0.143524,-12300.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0,-0.107629,-7900.0


### 19. Saving large datasets to CSVs

In [ ]:
# Saving dataframes to csv files is not the best way to store data
# It's better to use parquet, feather or pickle which are much more efficient

df = pd.read_csv('housing.csv')

df.to_parquet('housing.parquet')
df.to_feather('housing.feather')
df.to_pickle('housing.pkl')

### 20. Not leveraging pandas conditional formatting

In [23]:
df = pd.read_csv('housing.csv')

df.style.background_gradient(cmap='Blues')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


### 21. Not using suffixes when merging two dataframes

In [12]:
# Bad practice

df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 11, 12, 13, 14]})

df3 = df1.merge(df2, on=['A'])
df3

,A,B_x,B_y
0,1,4,10
1,2,5,11
2,3,6,12


In [13]:
# Good practice

df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 11, 12, 13, 14]})

df3 = df1.merge(df2, on=['A'], suffixes=('_left', '_right'))
df3

,A,B_left,B_right
0,1,4,10
1,2,5,11
2,3,6,12


### 22. Validating the merging of dataframes

In [14]:
# Good practice

df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 11, 12, 13, 14]})

df3 = df1.merge(df2, on=['A'], suffixes=('_left', '_right'), validate='one_to_one')

# validation options
# validate='one_to_one' will raise an error if there are duplicate values in the key column
# validate='one_to_many' will raise an error if there are duplicate values in the key column
# validate='many_to_one' will raise an error if there are duplicate values in the key column
# validate='many_to_many' will raise an error if there are duplicate values in the key column

df3 

,A,B_left,B_right
0,1,4,10
1,2,5,11
2,3,6,12


### 23. wrapping expression so they are readable 

In [ ]:
# To make the code more readable, it's better to write chained 
# Operations on separate lines surrounded by parentheses

# Bad practice
df_agg = df.groupby(['Grouping', 'Year'])['Time'].min().reset_index().fillna(0).sort_values(by='Time')

In [ ]:
# Good practice

df_agg = (
    df
    .groupby(['Grouping', 'Year'])['Time']
    .min()
    .reset_index()
    .fillna(0)
    .sort_values(by='Time')
)

### 24. Not using categorical datatypes

In [ ]:
# It's better to store categorical data as category dtype instead of strings

netflix_df['type'] = netflix_df['type'].astype('category')
netflix_df['type']

### 25. duplicating columns after concatenating

In [ ]:
# To avoid duplicating columns after concatenation 

df = pd.read_csv('housing.csv')
df_double = pd.concat([df, df], axis=1)
df_double = df_double.loc[:,~df_double.columns.duplicated()].copy()